In [2]:
import sqlite3
import pandas as pd
import pandas.io.sql as pd_sql

conn = sqlite3.connect("database.sqlite")
cursor = conn.cursor()

In [18]:
pd.read_sql_query('SELECT * FROM Match LIMIT 2',conn)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.2
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.6


In [19]:
pd.read_sql_query('SELECT * FROM sqlite_sequence LIMIT 2',conn)

,name,seq
0,Team,103916
1,Country,51958


In [20]:
pd.read_sql_query('SELECT * FROM Player_Attributes LIMIT 2',conn)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
1,2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8


In [21]:
pd.read_sql_query('SELECT * FROM Player LIMIT 2',conn)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146


In [22]:
pd.read_sql_query('SELECT * FROM League LIMIT 2',conn)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


In [23]:
pd.read_sql_query('SELECT * FROM Country LIMIT 2',conn)

,id,name
0,1,Belgium
1,1729,England


In [24]:
pd.read_sql_query('SELECT * FROM Team LIMIT 2',conn)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673,KRC Genk,GEN
1,2,9993,675,Beerschot AC,BAC


In [25]:
pd.read_sql_query('SELECT * FROM Team_Attributes LIMIT 2',conn)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover


1. Which team scored the most points when playing at home?  

    Real Madrid CF

In [39]:
pd.read_sql_query(
    'SELECT Team.team_long_name, SUM(home_team_goal) AS home_points FROM Match LEFT JOIN Team ON Team.team_api_id=Match.home_team_api_id GROUP BY team_long_name ORDER BY SUM(home_team_goal) DESC LIMIT 5',conn)

,team_long_name,home_points
0,Real Madrid CF,505
1,FC Barcelona,495
2,Celtic,389
3,FC Bayern Munich,382
4,PSV,370


2. Did this team also score the most points when playing away?

    No, FC Barcelona scored the most points when playing away.

In [41]:
pd.read_sql_query(
    'SELECT Team.team_long_name, SUM(away_team_goal) AS away_points FROM Match LEFT JOIN Team ON Team.team_api_id=Match.away_team_api_id GROUP BY team_long_name ORDER BY SUM(away_team_goal) DESC LIMIT 5',conn)

,team_long_name,away_points
0,FC Barcelona,354
1,Real Madrid CF,338
2,Celtic,306
3,Ajax,287
4,PSV,282


3. How many matches resulted in a tie?

    6596

In [64]:
pd.read_sql_query('SELECT COUNT(id) as Ties FROM Match WHERE home_team_goal=away_team_goal',conn)

,Ties
0,6596


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

    15 with the last name Smith, 18 with 'smith' anywhere in their name.

In [35]:
pd.read_sql_query("SELECT COUNT(player_name) FROM Player WHERE player_name LIKE '% Smith'",conn)

,COUNT(player_name)
0,15


In [33]:
pd.read_sql_query("SELECT COUNT(player_name) FROM Player WHERE player_name LIKE '%smith%'",conn)

,COUNT(player_name)
0,18


5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results.

    Median Tie Score = 1

In [69]:
query='''
SELECT ROUND(AVG(home_team_goal), 2) AS median  
FROM (  SELECT home_team_goal
        FROM ( SELECT home_team_goal,
           COUNT(*) OVER() AS row_count,
           ROW_NUMBER() OVER(ORDER BY home_team_goal=away_team_goal) AS row_number
    FROM [Match]
    WHERE home_team_goal=away_team_goal <> 0
  ) x
WHERE row_number IN ((row_count + 1)/2, (row_count + 2)/2)  
) y
;'''

In [70]:
pd.read_sql_query(query,conn)

,median
0,1.0


6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

    75% prefer right foot, 25% prefer left foot

In [57]:
pd.read_sql_query("SELECT (COUNT(preferred_foot)*100/(SELECT COUNT(*) FROM Player_Attributes)) AS Percent_Prefer_Right FROM Player_Attributes WHERE preferred_foot='right'",conn)


,Percent_Prefer_Right
0,75


To see all the Tables and Columns within each table:

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [4]:
def table_columns(db, table_name):
    curs = db.cursor()
    sql = "select * from %s where 1=0;" % table_name
    curs.execute(sql)
    return [d[0] for d in curs.description]

In [5]:
table_columns(conn, 'Player_Attributes')

['id',
 'player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [6]:
table_columns(conn, 'Player')

['id',
 'player_api_id',
 'player_name',
 'player_fifa_api_id',
 'birthday',
 'height',
 'weight']

In [7]:
table_columns(conn, 'Match')

['id',
 'country_id',
 'league_id',
 'season',
 'stage',
 'date',
 'match_api_id',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player

In [8]:
table_columns(conn, 'League')

['id', 'country_id', 'name']

In [9]:
table_columns(conn, 'Country')

['id', 'name']

In [10]:
table_columns(conn, 'Team')

['id', 'team_api_id', 'team_fifa_api_id', 'team_long_name', 'team_short_name']

In [11]:
table_columns(conn, 'Team_Attributes')

['id',
 'team_fifa_api_id',
 'team_api_id',
 'date',
 'buildUpPlaySpeed',
 'buildUpPlaySpeedClass',
 'buildUpPlayDribbling',
 'buildUpPlayDribblingClass',
 'buildUpPlayPassing',
 'buildUpPlayPassingClass',
 'buildUpPlayPositioningClass',
 'chanceCreationPassing',
 'chanceCreationPassingClass',
 'chanceCreationCrossing',
 'chanceCreationCrossingClass',
 'chanceCreationShooting',
 'chanceCreationShootingClass',
 'chanceCreationPositioningClass',
 'defencePressure',
 'defencePressureClass',
 'defenceAggression',
 'defenceAggressionClass',
 'defenceTeamWidth',
 'defenceTeamWidthClass',
 'defenceDefenderLineClass']